## Imports

In [6]:
import os
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import udf
# from config import sf_env_creds

from snowflake.core import Root
from snowflake.connector import connect
from snowflake.snowpark import functions as f
from snowflake.core.task import StoredProcedureCall, Task
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from datetime import timedelta


In [13]:
my_database = "DGMN_DEV_DB" 
my_schema = "STAGE_MLOPS"
warehouse = "DGMN_EXTRACT"

CONNECTION_PARAMETERS = {
"account": "gb59435.east-us-2.azure",
"user": "dgsvc_dbk_dsteam",
"password": "&U8i(O0p_{",
"database": my_database,
"warehouse": warehouse,
"schema": my_schema,
}

connection = connect(**CONNECTION_PARAMETERS)

api_root = Root(connection)
schema = api_root.databases[my_database].schemas[my_schema]


In [11]:
def update_order(session: Session) -> str:
    return 'updated order'

def settle_order(session: Session) -> str:
    return 'settled order'

In [14]:
dag_name = "dag_hello_world"
dag = DAG(dag_name, schedule=timedelta(minutes=5))
with dag:
    dag_task1 = DAGTask("update_order",StoredProcedureCall(update_order,stage_location="@DGMN_DEV_DB.STAGE_MLOPS.DAG",packages=["snowflake-snowpark-python"]),warehouse=warehouse)
    dag_task2 = DAGTask("settle_order",StoredProcedureCall(settle_order,stage_location="@DGMN_DEV_DB.STAGE_MLOPS.DAG",packages=["snowflake-snowpark-python"]),warehouse=warehouse)
    dag_task2 >> dag_task1

dag_op = DAGOperation(schema)
dag_op.deploy(dag, mode="orReplace")

APIError: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Keep-Alive': 'timeout=175', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'X-Snowflake-Request-ID': 'adb578a0-8fd5-42db-9e60-11fdd19d2106', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'Strict-Transport-Security': 'max-age=31536000', 'Date': 'Fri, 19 Jul 2024 16:42:16 GMT'})
HTTP response body: {
  "code" : "091447",
  "message" : "warehouse not specified and missing serverless task privilege to create task dag_hello_world. to create it as a user-managed task, specify a warehouse. to create it as a serverless task, execute the create task command with a role that has been granted the execute managed task account-level privilege.",
  "request_id" : "adb578a0-8fd5-42db-9e60-11fdd19d2106",
  "error_code" : "091447"
}


Run the DAG manually without waiting for the schedule (set to 5 minutes)

In [7]:
dag_op.run(dag) #Execute root task

In [8]:
dagiter = dag_op.iter_dags(like='dag_hello%')
for dag_name in dagiter:
    print(dag_name)

DAG_HELLO_WORLD
